<a href="https://colab.research.google.com/github/Soumya120/ML-Projects/blob/main/Pneumonia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
import os
import numpy as np
import pandas as pd

In [ ]:
train = r"C:\Users\ADMIN\Desktop\Internship project\chest_xray/train"
test = r"C:\Users\ADMIN\Desktop\Internship project\chest_xray/test"
val = r"C:\Users\ADMIN\Desktop\Internship project\chest_xray/val"

In [ ]:
batch_size=15
image_size=(120,120)

In [ ]:
train_datagen= ImageDataGenerator(rotation_range=15,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True,
                                 rescale=1./255) 

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary")

test_generator = test_datagen.flow_from_directory(
    test,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary")

validation_generator = test_datagen.flow_from_directory(
    val,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary")

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
model=Sequential()
model.add(Conv2D(30,(3,3),activation='relu',input_shape=(120,120,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(75,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(120,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(220,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 30)      840       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 30)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 59, 59, 30)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 75)        20325     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 75)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 28, 28, 75)        0

In [ ]:
model.fit_generator(train_generator,
                   steps_per_epoch=5216//15,
                   epochs=20,
                   verbose=1,
                   validation_data=test_generator,
                   validation_steps=624//15)

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_15516/2356288912.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/20
347/347 [==============================] - 257s 737ms/step - loss: 0.5617 - accuracy: 0.7543 - val_loss: 0.5654 - val_accuracy: 0.6569
Epoch 2/20
347/347 [==============================] - 207s 595ms/step - loss: 0.4098 - accuracy: 0.8191 - val_loss: 0.4716 - val_accuracy: 0.7382
Epoch 3/20
347/347 [==============================] - 204s 587ms/step - loss: 0.3352 - accuracy: 0.8641 - val_loss: 0.5970 - val_accuracy: 0.7285
Epoch 4/20
347/347 [==============================] - 203s 583ms/step - loss: 0.2829 - accuracy: 0.8850 - val_loss: 0.4747 - val_accuracy: 0.8098
Epoch 5/20
347/347 [==============================] - 219s 631ms/step - loss: 0.2468 - accuracy: 0.9046 - val_loss: 0.9639 - val_accuracy: 0.6780
Epoch 6/20
347/347 [==============================] - 180s 520ms/step - loss: 0.2501 - accuracy: 0.9106 - val_loss: 0.7019 - val_accuracy: 0.7398
Epoch 7/20
347/347 [==============================] - 178s 513ms/step - loss: 0.2318 - accuracy: 0.9131 - val_loss: 0.4888 -

In [ ]:
model.save('pneumonia.h5')

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

In [ ]:
model = load_model('pneumonia.h5')

In [ ]:
from skimage.transform import resize
import cv2
def detect(frame):
    try:
        img = resize(frame,(120,120))
        img = np.expand_dims(img,axis=0)
        prediction = model.predict(img)
        print(prediction)
        prediction = (model.predict(img) > 0.5).astype("int32")
        print(prediction)
    except AttributeError:
        print("Shape not found")
frame = cv2.imread(r"C:\Users\ADMIN\Desktop\Internship project\flask\uploads/test.jpg")
data = detect(frame)

[[0.9897635]]
[[1]]
